# Low-Level Design (LLD) of a LinkedIn System

Requirements

1. User Registration & Authentication: Sign-up, login, logout, password management.
2. User Profiles: Profile details (name, experience, education, skills), profile update.
3. Connections: Sending, accepting, and rejecting connection requests.
4. Messaging: Sending and receiving messages between connected users.
5. Job Postings: Employers post jobs, users browse and apply.
6. Search Functionality: Search users, companies, jobs with ranking.
7. Notifications: Real-time alerts for requests, messages, and job postings



In [2]:
import hashlib
import uuid

from datetime import date, datetime, timedelta
from enum import Enum
from typing import Optional, List, Set
from collections import defaultdict, deque

In [3]:
class InvalidDateException(Exception):
    """Raised when the provided date values are inconsistent."""

    def __init__(self, message: str):
        super().__init__(message)


class InvalidProficiencyLevelException(Exception):
    """Raised when an invalid proficiency level is provided."""

    def __init__(self, level: str):
        super().__init__(f"Invalid proficiency level: {level}. Choose from: Beginner, Intermediate, Expert.")


class Experience:

    def __init__(self, company: str, role: str, start_date: date, end_date: Optional[date] = None):
        if end_date and end_date < start_date:
            raise InvalidDateException("End date cannot be earlier than start date.")

        self.experience_id = str(uuid.uuid4())
        self.company = company
        self.role = role
        self.start_date = start_date
        self.end_date = end_date  # None if currently working

    def __repr__(self):
        return f"{self.role} at {self.company} ({self.start_date} - {self.end_date or 'Present'})"


class Education:

    def __init__(self, institution: str, degree: str, start_year: date, end_year: date):
        if start_year > end_year:
            raise InvalidDateException("Start year cannot be after end year.")

        self.education_id = str(uuid.uuid4())
        self.institution = institution
        self.degree = degree
        self.start_year = start_year
        self.end_year = end_year

    def __repr__(self):
        return f"{self.degree} from {self.institution} ({self.start_year} - {self.end_year})"


class ProficiencyLevel(Enum):
    BEGINNER = 'Beginner'
    INTERMEDIATE = 'Intermediate'
    EXPERT = 'Expert'


class Skill:

    def __init__(self, name: str, proficiency_level: ProficiencyLevel = ProficiencyLevel.BEGINNER):
        self.skill_id = str(uuid.uuid4())
        self.name = name
        self.proficiency_level = proficiency_level

    def __repr__(self):
        return f"{self.name} ({self.proficiency_level.value})"

    @classmethod
    def get_proficiency_level(cls, level: str) -> ProficiencyLevel:
        try:
            return ProficiencyLevel(level.title())
        except ValueError:
            raise InvalidProficiencyLevelException(level)

In [4]:
class BaseUserProfile:
    
    def __init__(self, headline: Optional[str] = None, summary: Optional[str] = None):
        self.profile_id = str(uuid.uuid4())
        self.headline = headline
        self.summary = summary

    def update_profile(
        self,
        headline: Optional[str] = None,
        summary: Optional[str] = None
    ):
        """Updates basic profile details"""
        if headline:
            self.headline = headline

        if summary:
            self.summary = summary
            
    def display_profile(self):
        """Returns the user profile as a dictionary"""
        return {
            "Headline": self.headline,
            "Summary": self.summary,
        }


class UserProfile(BaseUserProfile):

    def __init__(self, headline: Optional[str] = None, summary: Optional[str] = None):
        super().__init__(headline, summary)
        self.experience: Optional[Experience] = []     # List of Experience objects
        self.education: Optional[Education] = []      # List of Education objects
        self.skills: Optional[Skill] = []         # List of Skill objects

    def add_experience(
        self,
        company: str,
        role: str,
        start_date: str,
        end_date: Optional[date] = None
    ):
        """Adds a new experience entry"""
        self.experience.append(Experience(company, role, start_date, end_date))

    def add_education(
        self,
        institution: str,
        degree: str,
        start_year: date,
        end_year: date
    ):
        """Adds a new education entry"""
        self.education.append(
            Education(institution, degree, start_year, end_year)
        )

    def add_skill(self, name: str, proficiency_level: ProficiencyLevel):
        """Adds a new skill"""
        self.skills.append(Skill(name, proficiency_level))

    def display_profile(self):
        """Returns the user profile as a dictionary"""
        profile_data =  super().display_profile()
        profile_data.update({
            "Experience": [str(exp) for exp in self.experience],
            "Education": [str(edu) for edu in self.education],
            "Skills": [str(skill) for skill in self.skills],
        })
        return profile_data

In [5]:
class Role(Enum):
    USER = "User"
    EMPLOYER = "Employer"
    

class BaseUser:
    """A shared base class for User and Employer, handling authentication."""

    def __init__(self, name: str, email: str, password: str, role: Role):
        self.user_id = str(uuid.uuid4())
        self.name = name
        self.email = email.lower()
        self.role = role
        self._password_hash = self._hash_password(password)
        self.session_token = None  # Stores login session

    def _hash_password(self, password: str):
        """Hashes the password using SHA-256"""
        return hashlib.sha256(password.encode()).hexdigest()

    def check_password(self, password: str):
        """Checks if the entered password matches the stored hash"""
        return self._password_hash == self._hash_password(password)

    def generate_session_token(self):
        """Generates a session token when user logs in"""
        self.session_token = str(uuid.uuid4())

    def clear_session(self):
        """Clears the session on logout"""
        self.session_token = None

    def __repr__(self):
        return f"{self.name}"


class User(BaseUser):
    """Extends BaseUser, adding professional profile and connections."""

    def __init__(self, name: str, email: str, password: str):
        super().__init__(name, email, password, Role.USER)
        self.profile = UserProfile()
        self.connections: Set[str] = set()  # Stores user IDs of accepted connections

        
    def display_profile(self):
        """Displays the profile with the employer's name as the company name."""
        return self.profile.display_profile()

In [6]:
class EmployerProfile(BaseUserProfile):

    def __init__(self, headline: Optional[str] = None, summary: Optional[str] = None, founded_date: Optional[date] = None):
        super().__init__(headline, summary)
        self.founded_date = founded_date
        self.posted_jobs: List['Job'] = []

    def display_profile(self, company_name: str):
        """Returns the employer profile as a dictionary"""
        profile_data = super().display_profile()
        profile_data.update({
            "Company Name": company_name,
            "Founded": self.founded_date.year if self.founded_date else "N/A",
            "Posted Jobs": [job.title for job in self.posted_jobs]
        })
        return profile_data

    def update_profile(self, headline: Optional[str] = None, summary: Optional[str] = None, founded_date: Optional[date] = None):
        """Updates the employer profile details"""
        super().update_profile(headline, summary)
        if founded_date:
            self.founded_date = founded_date


class Employer(BaseUser):
    """Extends BaseUser, using employer name as company name."""

    def __init__(self, name: str, email: str, password: str, founded_date: Optional[date] = None):
        super().__init__(name, email, password, Role.EMPLOYER)
        self.profile = EmployerProfile(headline=None, summary=None, founded_date=founded_date)  # Fix: Correct argument passing

    def display_profile(self):
        """Displays the profile with the employer's name as the company name."""
        return self.profile.display_profile(self.name)


In [7]:
class UserManager:
    
    def __init__(self):
        self.users = {}  # Stores users and employers by email
        self.active_sessions = {}  # Tracks logged-in users/employers by session token
        
    def _get_authenticated_user(self, session_token: str) -> BaseUser:
        """Helper method to retrieve the authenticated user or employer."""
        user = self.active_sessions.get(session_token)
        if not user:
            raise UserNotLoggedInException()
        return user

    def register_user(self, name: str, email: str, password: str):
        """Registers a new job seeker (user)"""
        email = email.lower()
        if email in self.users:
            raise UserAlreadyExistsException(email)

        user = User(name, email, password)
        self.users[email] = user
        return user

    def register_employer(self, company_name: str, email: str, password: str, founded_date: Optional[date] = None):
        """Registers a new employer (company)"""
        email = email.lower()
        if email in self.users:
            raise UserAlreadyExistsException(email)

        employer = Employer(company_name, email, password, founded_date)
        self.users[email] = employer
        return employer

    def login_user(self, email: str, password: str):
        """Authenticates a user or employer and creates a session"""
        email = email.lower()
        user = self.users.get(email)
        if not user or not user.check_password(password):
            raise InvalidCredentialsException()

        if user.session_token is not None:
            raise UserAlreadyLoggedInException(user.email)

        user.generate_session_token()
        self.active_sessions[user.session_token] = user
        print(f"✅ {user.name} has logged in successfully!")
        return user.session_token

    def logout_user(self, session_token: str):
        """Logs out a user or employer and removes the session"""
        user = self._get_authenticated_user(session_token)
        user.clear_session()
        print(f"🔓 {user.name} has logged out successfully!")

    def update_user_profile(self, session_token: str, headline: Optional[str] = None, summary: Optional[str] = None):
        """Updates only user profiles"""
        user = self._get_authenticated_user(session_token)
        if not isinstance(user.profile, UserProfile):
            raise PermissionDeniedException("update user profile")
        user.profile.update_profile(headline, summary)
        print(f"✅ {user.name}'s profile updated successfully!")

    def update_employer_profile(self, session_token: str, headline: Optional[str] = None, summary: Optional[str] = None, founded_date: Optional[date] = None):
        """Updates only employer profiles"""
        user = self._get_authenticated_user(session_token)
        if not isinstance(user.profile, EmployerProfile):
            raise PermissionDeniedException("update employer profile")
        user.profile.update_profile(headline, summary, founded_date)
        print(f"✅ {user.name}'s company profile updated successfully!")

    def view_profile(self, session_token: str):
        """Returns user or employer profile"""
        user = self._get_authenticated_user(session_token)
        return user.display_profile()

    ### --- User Functions --- ###
    
    def add_experience(self, session_token: str, company: str, role: str, start_date: date, end_date: Optional[date] = None):
        """Adds experience to a user's profile"""
        user = self._get_authenticated_user(session_token)
        if not isinstance(user.profile, UserProfile):
            raise PermissionDeniedException("add experience")
        user.profile.add_experience(company, role, start_date, end_date)
        print(f"✅ Experience added for {user.name}!")

    def add_education(self, session_token: str, institution: str, degree: str, start_year: date, end_year: date):
        """Adds education to a user's profile"""
        user = self._get_authenticated_user(session_token)
        if not isinstance(user.profile, UserProfile):
            raise PermissionDeniedException("add education")
        user.profile.add_education(institution, degree, start_year, end_year)
        print(f"✅ Education added for {user.name}!")

    def add_skill(self, session_token: str, name: str, proficiency_level: str):
        """Adds skill to a user's profile"""
        user = self._get_authenticated_user(session_token)
        if not isinstance(user.profile, UserProfile):
            raise PermissionDeniedException("add skill")
        proficiency_level = Skill.get_proficiency_level(proficiency_level)
        user.profile.add_skill(name, proficiency_level)
        print(f"✅ Skill added for {user.name}!")


class UserAlreadyExistsException(Exception):
    """Raised when trying to register an already existing email."""

    def __init__(self, email):
        super().__init__(f"{email} Email already registered!")


class InvalidCredentialsException(Exception):
    """Raised when login credentials are incorrect."""

    def __init__(self):
        super().__init__("Invalid email or password!")


class UserNotLoggedInException(Exception):
    """Raised when performing an action that requires login."""

    def __init__(self):
        super().__init__("Invalid session or user not logged in!")


class UserAlreadyLoggedInException(Exception):
    """Raised when performing an action that requires login."""

    def __init__(self, email):
        super().__init__(f"User {email} already LoggedIn.")
        
class PermissionDeniedException(Exception):
    
    def __init__(self, action: str):
        super().__init__(f"User doesn't has permission to perform {action} action.")

In [8]:
class ConnectionRequestProccesedException(Exception):
    """Raised when performing an action that requires login."""

    def __init__(self, request_id: str):
        super().__init__(f"connection request {request_id} has been already proccesed.")


class ConnectionRequestInvalidException(Exception):

    def __init__(self):
        super().__init__("Cannot send a connection request to yourself.")


class DuplicateConnectionRequestException(Exception):

    def __init__(self):
        super().__init__("A pending connection request already exists.")


class ConnectionStatus(Enum):
    PENDING = "Pending"
    ACCEPTED = "Accepted"
    REJECTED = "Rejected"


class ConnectionRequest:
    def __init__(self, sender: User, receiver: User):
        self.request_id = str(uuid.uuid4())
        self.sender = sender
        self.receiver = receiver
        self.status = ConnectionStatus.PENDING

    def accept(self):
        """Accept the connection request and update both users' connection lists."""
        if self.status != ConnectionStatus.PENDING:
            raise ConnectionRequestProccesedException(self.request_id)

        self.status = ConnectionStatus.ACCEPTED
        self.sender.connections.add(self.receiver)
        self.receiver.connections.add(self.sender)

    def reject(self):
        """Reject the connection request."""
        if self.status != ConnectionStatus.PENDING:
            raise ConnectionRequestProccesedException(self.request_id)

        self.status = ConnectionStatus.REJECTED

    def __repr__(self):
        return f"ConnectionRequest({self.sender.name} -> {self.receiver.name}, Status={self.status.value})"

In [9]:
class ConnectionManager:

    def __init__(self):
        self.requests: List[ConnectionRequest] = []  # Store all connection requests

    def send_request(self, sender: User, receiver: User):
        """Send a new connection request if not already connected or pending."""
        if sender.user_id == receiver.user_id:
            raise ConnectionRequestInvalidException()

        for req in self.requests:
            if req.sender == sender and req.receiver == receiver and req.status == ConnectionStatus.PENDING:
                raise DuplicateConnectionRequestException()

        request = ConnectionRequest(sender, receiver)
        self.requests.append(request)
        return request

    def view_requests(self, receiver: User):
        """View all pending connection requests for a user."""
        return [req for req in self.requests if req.receiver == receiver and req.status == ConnectionStatus.PENDING]

    def get_connections(self, user: User):
        """Retrieve all accepted connections for a user."""
        return user.connections
    
    def accept_request(self, user: User, request_id: str) -> ConnectionRequest:
        """Accept a connection request by request ID."""
        request = self._get_request_by_id(request_id)
        if request.receiver != user:
            raise PermissionDeniedException('ACCEPT_REQUEST')
        request.accept()
        return request

    def reject_request(self, user: User, request_id: str):
        """Reject a connection request by request ID."""
        request = self._get_request_by_id(request_id)
        if request.receiver != user:
            raise PermissionDeniedException('REJECT_REQUEST')
        request.reject()

    def _get_request_by_id(self, request_id: str):
        """Helper method to get a connection request by ID."""
        for req in self.requests:
            if req.request_id == request_id:
                return req
        raise ConnectionRequestProccesedException(request_id)

In [10]:
class MessagePermissionException(Exception):
    """Raised when a user tries to send a message to a non-connection."""

    def __init__(self, sender, receiver):
        super().__init__(f"User '{sender.name}' cannot message '{receiver.name}' as they are not connected.")


class MessageStatus(Enum):
    SENT = "Sent"
    RECEIVED = "Received"
    READ = "Read"


class Message:

    def __init__(self, sender: User, receiver: User, content: str):
        self.message_id = str(uuid.uuid4())
        self.sender = sender
        self.receiver = receiver
        self.content = content
        self.timestamp = datetime.utcnow()
        self.delivered_at = None  # When received
        self.read_at = None  # When read
        self.status = MessageStatus.SENT

    def mark_received(self):
        if self.status == MessageStatus.SENT:
            self.status = MessageStatus.RECEIVED
            self.delivered_at = datetime.utcnow()

    def mark_read(self):
        if self.status == MessageStatus.RECEIVED:
            self.status = MessageStatus.READ
            self.read_at = datetime.utcnow()

    def __repr__(self):
        return f"Message({self.sender.name} -> {self.receiver.name}, Status={self.status.value}, Time={self.timestamp})"

In [11]:
class MessageManager:

    def __init__(self):
        self.sent_messages = defaultdict(deque)
        self.received_messages = defaultdict(deque)
        self.message_lookup = {}
        self.last_read_message = {}

    def send_message(self, sender: User, receiver: User, content: str):
        if receiver not in sender.connections:
            raise MessagePermissionException(sender, receiver)

        message = Message(sender, receiver, content)
        self.sent_messages[sender.user_id].append(message)
        self.received_messages[receiver.user_id].append(message)
        self.message_lookup[message.message_id] = message
        self.mark_as_received(message.message_id, receiver)
        return message

    def mark_as_received(self, message_id: str, receiver: User):
        if message_id in self.message_lookup:
            msg = self.message_lookup[message_id]
            if msg.receiver == receiver and msg.status == MessageStatus.SENT:
                msg.mark_received()

    def mark_as_read(self, sender: User, receiver: User):
        unread_messages = [
            msg for msg in self.received_messages[receiver.user_id]
            if msg.sender == sender and msg.status == MessageStatus.RECEIVED
        ]
        for msg in unread_messages:
            msg.mark_read()

        if unread_messages:
            self.last_read_message[receiver.user_id] = unread_messages[-1].message_id

    def get_chat_history(self, user1: User, user2: User):
        return [
            msg for msg in sorted(
                self.sent_messages[user1.user_id] + self.received_messages[user1.user_id],
                key=lambda m: m.timestamp
            ) if msg.sender == user2 or msg.receiver == user2
        ]

    def get_unread_message_count(self, receiver: User):
        return sum(
            1 for msg in self.received_messages[receiver.user_id] if msg.status != MessageStatus.READ
        )

    def remove_old_messages(self, days: Optional[int] = 30):
        threshold = datetime.utcnow() - timedelta(days=days)

        for user_id in list(self.sent_messages.keys()):
            self.sent_messages[user_id] = deque(
                msg for msg in self.sent_messages[user_id] if msg.timestamp > threshold
            )

        for user_id in list(self.received_messages.keys()):
            self.received_messages[user_id] = deque(
                msg for msg in self.received_messages[user_id] if msg.timestamp > threshold
            )

In [12]:
class JobNotFoundException(Exception):
    """Raised when a job ID is not found."""
    def __init__(self, job_id):
        super().__init__(f"Job with ID {job_id} not found.")


class JobAlreadyAppliedException(Exception):
    """Raised when a user tries to apply for the same job multiple times."""
    def __init__(self, user, job):
        super().__init__(f"User {user.name} has already applied for job {job.title}.")


class InvalidJobApplicationException(Exception):
    """Raised when an invalid job application is attempted."""
    def __init__(self, message="Invalid job application attempt."):
        super().__init__(message)


class InvalidPincodeException(Exception):
    """Raised when an invalid job application is attempted."""
    def __init__(self, message="Pincode must be a non-negative integer"):
        super().__init__(message)


class Location:
    def __init__(self, city: str, state: str, country: str, pincode: int):
        self.city = city
        self.state = state
        self.country = country
        self.pincode = self.validate_pincode(pincode)

    @staticmethod
    def validate_pincode(pincode):
        if not isinstance(pincode, int) or pincode < 0:
            raise InvalidPincodeException()
        return pincode

    def __repr__(self):
        return f"Location(city='{self.city}', state='{self.state}', country='{self.country}', pincode={self.pincode})"

    def __str__(self):
        return f"{self.city}, {self.state}, {self.country} - {self.pincode}"


class JobLocation(Location):

    def __repr__(self):
        return f"{self.city}, {self.state}, {self.country} - {self.pincode}"


class JobType(Enum):
    FULL_TIME = 'Full-Time'
    PART_TIME = 'Part-Time'
    CONTRACT = "Contract"
    INTERN = "Intern"


class Job:
    def __init__(
        self,
        employer: Employer,
        title: str,
        description: str,
        location: JobLocation,
        job_type: JobType,
        salary: Optional[float] = None
    ):
        self.job_id = str(uuid.uuid4())
        self.employer = employer  # Store only the employer ID
        self.title = title
        self.description = description
        self.location = location
        self.job_type = job_type
        self.salary = salary
        self.posted_at = datetime.utcnow()

    def __repr__(self):
        return f"Job({self.title} at {self.location}, Type={self.job_type.value})"
    

class JobApplicationStatus(Enum):
    PENDING = "Pending"
    ACCEPTED = "Accepted"
    REJECTED = "Rejected"


class JobApplication:
    def __init__(self, user: User, job: Job):
        self.application_id = str(uuid.uuid4())
        self.user_id = user.user_id  # Store only User ID
        self.job_id = job.job_id  # Store only Job ID
        self.status = JobApplicationStatus.PENDING
        self.applied_at = datetime.utcnow()

    def update_status(self, new_status: JobApplicationStatus):
        """Update application status"""
        self.status = new_status

    def application_age(self):
        """Returns the age of the application in days"""
        return (datetime.utcnow() - self.applied_at).days

    def __repr__(self):
        return f"Application(User: {self.user_id} -> Job: {self.job_id}, Status={self.status.value})"



In [13]:
class JobManager:
    def __init__(self):
        self.jobs: Dict[str, Job] = {}  # Stores jobs by job_id
        self.applications: Dict[str, List[JobApplication]] = {}  # Maps job_id to applications

    def post_job(self, employer: Employer, title: str, description: str, location: JobLocation, job_type: JobType, salary: Optional[float] = None) -> Job:
        """Allows an employer to post a job."""
        if not employer.session_token:
            raise UserNotLoggedInException()
        
        if not isinstance(employer, Employer):
            raise PermissionDeniedException('POST_JOB')

        job = Job(employer, title, description, location, job_type, salary)
        self.jobs[job.job_id] = job
        employer.profile.posted_jobs.append(job)
        return job

    def get_job(self, job_id: str) -> Job:
        """Retrieves a job by its ID."""
        if job_id not in self.jobs:
            raise JobNotFoundException(job_id)
        return self.jobs[job_id]

    def get_employer_jobs(self, employer: Employer) -> List[Job]:
        """Retrieves all jobs posted by a specific employer."""
        if not isinstance(employer, Employer):
            raise PermissionDeniedException('GTE_JOB')
        return [job for job in self.jobs.values() if job.employer == employer]
    
    def remove_job(self, employer: Employer, job_id: str):
        """Deletes a job and removes related applications."""
        if job_id not in self.jobs:
            raise JobNotFoundException(job_id)

        job = self.jobs[job_id]
        if not isinstance(employer, Employer) or job.employer != employer:
            raise PermissionDeniedException('REMOVE_JOB')

        # Remove the job from employer's profile
        employer.profile.posted_jobs.remove(job)

        # Remove job and related applications
        del self.jobs[job_id]
        self.applications.pop(job_id, None)

    def apply_for_job(self, user: User, job_id: str) -> JobApplication:
        """Allows a user to apply for a job."""
        if not isinstance(user, User):
            raise PermissionDeniedException('APPLY_JOB')

        
        if job_id not in self.jobs:
            raise JobNotFoundException(job_id)
        
        job = self.jobs[job_id]
        if job_id in self.applications and any(app.user_id == user.user_id for app in self.applications[job_id]):
            raise JobAlreadyAppliedException(user, job)

        application = JobApplication(user, job)
        self.applications.setdefault(job_id, []).append(application)
        return application

    def get_applications(self, employer: Employer, job_id: str) -> List[JobApplication]:
        """Retrieves all applications for a job."""
        if not isinstance(employer, Employer):
            raise PermissionDeniedException('REMOVE_JOB')
        
        if job_id not in self.jobs:
            raise JobNotFoundException(job_id)
        
        job = self.jobs[job_id]
        
        if job.employer != employer:
            raise PermissionDeniedException('REMOVE_JOB')
        
        return self.applications.get(job_id, [])

    def get_user_applications(self, user: User) -> List[JobApplication]:
        """Retrieves all applications submitted by a user."""
        return [app for apps in self.applications.values() for app in apps if app.user_id == user.user_id]

    def update_application_status(self, employer: Employer, application_id: str, new_status: JobApplicationStatus):
        """Updates the status of a job application."""
        if not isinstance(employer, Employer):
            raise PermissionDeniedException('UPDATE_APPLICATION_STATUS')
        
        for job_id, applications in self.applications.items():
            job = self.jobs.get(job_id)
            if not job or job.employer != employer:
                continue

            for app in applications:
                if app.application_id == application_id:
                    app.update_status(new_status)
                    return app
        raise InvalidJobApplicationException(f"Application ID {application_id} not found.")

    def __repr__(self):
        return f"JobManager({len(self.jobs)} jobs, {sum(len(apps) for apps in self.applications.values())} applications)"

In [14]:
class SearchType(Enum):
    USER = "User"
    COMPANY = "Company"
    JOB = "Job"


class SearchManager:

    def __init__(self, user_manager: UserManager, job_manager: JobManager):
        self.users = user_manager.users
        self.jobs = job_manager.jobs.values()


    def search(self, query, search_type=SearchType.USER):
        """Search for users, companies, or jobs based on a query."""
        query = query.lower()

        if search_type == SearchType.USER:
            results = [(user, self._rank(query, user.name)) for _, user in self.users.items() if user.role == Role.USER]
        elif search_type == SearchType.COMPANY:
            results = [(company, self._rank(query, company.name)) for _, company in self.users.items() if company.role == Role.EMPLOYER]
        elif search_type == SearchType.JOB:
            results = [
                (job, max(self._rank(query, job.title), self._rank(query, job.description)))
                for job in self.jobs
            ]
        else:
            return []

        # Sort results based on ranking (higher score first)
        return sorted(results, key=lambda x: x[1], reverse=True)

    def _rank(self, query, text):
        """Basic ranking function: counts how many times the query appears in text."""
        return text.lower().count(query)

In [15]:
class NotificationType(Enum):
    CONNECTION_REQUEST = "Connection Request"
    MESSAGE = "Message"
    JOB_POSTING = "Job Posting"


class Notification:
    def __init__(self, recipient: User, message: str, notif_type: NotificationType):
        self.notification_id = str(uuid.uuid4())
        self.recipient = recipient
        self.message = message
        self.type = notif_type
        self.timestamp = datetime.utcnow()
        self.read = False  # Unread by default

    def mark_as_read(self):
        """Mark notification as read."""
        self.read = True

    def __repr__(self):
        status = "✅ Read" if self.read else "🔔 Unread"
        return f"Notification({self.type.value}: {self.message} - {status}, {self.timestamp})"


class NotificationManager:

    def __init__(self):
        self.notifications = defaultdict(list)

    def send_notification(self, recipient: User, message: str, notif_type: NotificationType) -> Notification:
        """Create and store a new notification."""
        notification = Notification(recipient, message, notif_type)
        self.notifications[recipient.user_id].append(notification)
        return notification

    def get_unread_notifications(self, user: User) -> List[Notification]:
        """Fetch unread notifications for a user."""
        return [n for n in self.notifications[user.user_id] if not n.read]

    def mark_notifications_as_read(self, user: User):
        """Mark all notifications as read for a user."""
        for notif in self.notifications[user.user_id]:
            notif.mark_as_read()

In [16]:
class UnifiedManager:

    def __init__(self):
        self.user_manager = UserManager()
        self.connection_manager = ConnectionManager()
        self.message_manager = MessageManager()
        self.job_manager = JobManager()
        self.search_manager = SearchManager(self.user_manager, self.job_manager)
        self.notification_manager = NotificationManager()

    def _get_authenticated_user(self, session_token: str):
        """Helper method to get authenticated user."""
        user = self.user_manager.active_sessions.get(session_token)
        if not user:
            raise UserNotLoggedInException()
        return user

    def register_user(self, name: str, email: str, password: str):
        """Registers a new user using UserManager."""
        return self.user_manager.register_user(name, email, password)
    
    def register_employer(self, name: str, email: str, password: str, founded_date: Optional[date] = None):
        """Registers a new employer using UserManager."""
        return self.user_manager.register_employer(name, email, password, founded_date)

    def login_user(self, email: str, password: str):
        """Logs in a user using UserManager."""
        return self.user_manager.login_user(email, password)

    def logout_user(self, session_token: str):
        """Logs out a user using UserManager."""
        user = self._get_authenticated_user(session_token)
        self.user_manager.logout_user(session_token)

    def update_user_profile(self, session_token: str, headline: Optional[str] = None, summary: Optional[str] = None):
        """Updates the profile using UserManager."""
        user = self._get_authenticated_user(session_token)
        self.user_manager.update_user_profile(session_token, headline, summary)
        
    def update_employer_profile(self, session_token: str, headline: Optional[str] = None, summary: Optional[str] = None, founded_date: Optional[date] = None):
        """Updates the profile using UserManager."""
        user = self._get_authenticated_user(session_token)
        self.user_manager.update_employer_profile(session_token, headline, summary, founded_date)

    def add_experience(self, session_token: str, company: str, role: str, start_date: date, end_date: Optional[date] = None):
        """Adds experience to the user's profile using UserManager."""
        user = self._get_authenticated_user(session_token)
        self.user_manager.add_experience(session_token, company, role, start_date, end_date)

    def add_education(self, session_token: str, institution: str, degree: str, start_year: date, end_year: date):
        """Adds education to the user's profile using UserManager."""
        user = self._get_authenticated_user(session_token)
        self.user_manager.add_education(session_token, institution, degree, start_year, end_year)

    def add_skill(self, session_token: str, name: str, proficiency_level: str):
        """Adds skill to the user's profile using UserManager."""
        user = self._get_authenticated_user(session_token)
        self.user_manager.add_skill(session_token, name, proficiency_level)

    def view_profile(self, session_token: str):
        """Views the user's profile using UserManager."""
        user = self._get_authenticated_user(session_token)
        return self.user_manager.view_profile(session_token)

    # Connection Features #

    def send_connection_request(self, session_token: str, receiver_email: str):
        """Send a connection request using ConnectionManager."""
        sender = self._get_authenticated_user(session_token)
        receiver = self.user_manager.users.get(receiver_email.lower())
        if not receiver:
            raise UserNotFoundException()
        request = self.connection_manager.send_request(sender, receiver)
        
        self.notification_manager.send_notification(
            recipient=receiver,
            message=f"{sender.name} has sent you a connection request.",
            notif_type=NotificationType.CONNECTION_REQUEST
        )
        
        return request

    def view_connection_requests(self, session_token: str):
        """Views all pending connection requests for the logged-in user."""
        sender = self._get_authenticated_user(session_token)
        return self.connection_manager.view_requests(sender)

    def get_connections(self, session_token: str):
        """Gets the accepted connections for the logged-in user."""
        user = self._get_authenticated_user(session_token)
        return self.connection_manager.get_connections(user)

    def accept_connection_request(self, session_token: str, request_id: str):
        """Accept a pending connection request."""
        user = self._get_authenticated_user(session_token)
        request = self.connection_manager.accept_request(user, request_id)
        
        self.notification_manager.send_notification(
            recipient=request.sender,
            message=f"{user.name} has accepted your connection request.",
            notif_type=NotificationType.CONNECTION_REQUEST
        )

    def reject_connection_request(self, session_token: str, request_id: str):
        """Reject a pending connection request."""
        user = self._get_authenticated_user(session_token)
        self.connection_manager.reject_request(user, request_id)
        

    # Message Features #

    def send_message(self, session_token: str, receiver_email: str, content: str):
        """Sends a message to a connected user."""
        sender = self._get_authenticated_user(session_token)
        receiver = self.user_manager.users.get(receiver_email.lower())

        if not receiver:
            raise UserNotFoundException()

        message = self.message_manager.send_message(sender, receiver, content)
        
        self.notification_manager.send_notification(
            recipient=receiver,
            message=f"You have a new message from {sender.name}.",
            notif_type=NotificationType.MESSAGE
        )
        
        return message

    def mark_message_as_received(self, session_token: str, message_id: str):
        """Marks a message as received."""
        receiver = self._get_authenticated_user(session_token)
        self.message_manager.mark_as_received(message_id, receiver)

    def mark_messages_as_read(self, session_token: str, sender_email: str):
        """Marks all messages from a specific sender as read."""
        receiver = self._get_authenticated_user(session_token)
        sender = self.user_manager.users.get(sender_email.lower())

        if not sender:
            raise UserNotFoundException()

        self.message_manager.mark_as_read(sender, receiver)

    def get_chat_history(self, session_token: str, user_email: str):
        """Retrieves chat history with another user."""
        user = self._get_authenticated_user(session_token)
        other_user = self.user_manager.users.get(user_email.lower())

        if not other_user:
            raise UserNotFoundException()

        return self.message_manager.get_chat_history(user, other_user)

    def get_unread_message_count(self, session_token: str):
        """Gets the number of unread messages for the logged-in user."""
        receiver = self._get_authenticated_user(session_token)
        return self.message_manager.get_unread_message_count(receiver)

    def remove_old_messages(self, session_token: str, days: int = 30):
        """Removes messages older than a specified number of days."""
        user = self._get_authenticated_user(session_token)
        self.message_manager.remove_old_messages(days)
    
    # Job Features #
    
    def post_job(self, session_token: str, title: str, description: str, location: JobLocation, job_type: JobType, salary: Optional[float] = None):
        """Allows an employer to post a job."""
        employer = self._get_authenticated_user(session_token)
        job = self.job_manager.post_job(employer, title, description, location, job_type, salary)    
        return job
    
    def get_job(self, session_token: str, job_id: str):
        """Retrieves a job by its ID."""
        user = self._get_authenticated_user(session_token)
        return self.job_manager.get_job(job_id)
    
    def remove_job(self, session_token: str, job_id: str):
        employer = self._get_authenticated_user(session_token)
        return self.job_manager.remove_job(employer, job_id)
    
    def apply_for_job(self, session_token: str, job_id: str):
        """Allows a user to apply for a job."""
        user = self._get_authenticated_user(session_token)
        return self.job_manager.apply_for_job(user, job_id)
    
    def get_job_applications(self, session_token: str, job_id: str):
        """Retrieves all applications for a job."""
        employer = self._get_authenticated_user(session_token)
        return self.job_manager.get_applications(employer, job_id)
    
    def get_user_applications(self, session_token: str):
        """Retrieves all applications submitted by the logged-in user."""
        user = self._get_authenticated_user(session_token)
        return self.job_manager.get_user_applications(user)
    
    def update_application_status(self, session_token: str, application_id: str, new_status: JobApplicationStatus):
        """Updates the status of a job application."""
        employer = self._get_authenticated_user(session_token)
        return self.job_manager.update_application_status(employer, application_id, new_status)
    
    # Search Features #
    
    def search(self, session_token: str, query: str, search_type: SearchType = SearchType.USER):
        """Search for users, companies, or jobs based on a query."""
        user = self._get_authenticated_user(session_token)
        return self.search_manager.search(query, search_type)
    
    # Notification Methods

    def get_unread_notifications(self, session_token: str):
        """Retrieve unread notifications for the logged-in user."""
        user = self._get_authenticated_user(session_token)
        return self.notification_manager.get_unread_notifications(user)

    def mark_notifications_as_read(self, session_token: str):
        """Mark all notifications as read for the logged-in user."""
        user = self._get_authenticated_user(session_token)
        self.notification_manager.mark_notifications_as_read(user)

In [17]:
# Initialize UnifiedManager
unified_manager = UnifiedManager()

# User-related operations
alice = unified_manager.register_user("Alice", "alice@example.com", "password123")
bob = unified_manager.register_user("Bob", "bob@example.com", "password123")

# Login users
alice_session = unified_manager.login_user("alice@example.com", "password123")
bob_session = unified_manager.login_user("bob@example.com", "password123")

# Update user profile
unified_manager.update_user_profile(alice_session, headline="Software Engineer", summary="Experienced in Python, Django, and AI")


# View user profile
print(unified_manager.view_profile(alice_session))

# Send a connection request
request_1 = unified_manager.send_connection_request(alice_session, bob.email)

# Get notification
unread_notifications = unified_manager.get_unread_notifications(bob_session)
print(unread_notifications)


# Connection-related operations
pending_requests = unified_manager.view_connection_requests(bob_session)
print(pending_requests)

# Get connections for the user
connections = unified_manager.get_connections(bob_session)
print(connections)

# Accept request
unified_manager.accept_connection_request(bob_session, request_1.request_id)

# Get connections for the user
connections = unified_manager.get_connections(bob_session)
print(connections)

connections = unified_manager.get_connections(alice_session)
print(connections)


# Bob checks unread messages
unread_count = unified_manager.get_unread_message_count(bob_session)
print(f"Bob's unread messages: {unread_count}")

# Alice sends a message to Bob
message = unified_manager.send_message(alice_session, "bob@example.com", "Hello, Bob!")


# Get notification
unread_notifications = unified_manager.get_unread_notifications(bob_session)
print(unread_notifications)

# Bob checks unread messages
unread_count = unified_manager.get_unread_message_count(bob_session)
print(f"Bob's unread messages: {unread_count}")

# Bob reads messages from Alice
unified_manager.mark_messages_as_read(bob_session, "alice@example.com")

# Fetch chat history
chat_history = unified_manager.get_chat_history(alice_session, "bob@example.com")
for msg in chat_history:
    print(msg)


# Register Employer
google = unified_manager.register_employer("Google", "test@goole.com", "password123", date(2012, 5, 1))

# Login Employer
google_session = unified_manager.login_user("test@goole.com", "password123")

# Update Employer profile
unified_manager.update_employer_profile(google_session, headline="Tech Firm", summary="Technology based firm")

# View Employer profile
print(unified_manager.view_profile(google_session))

# Post job
job = unified_manager.post_job(google_session, "Backend Developer", "Develop APIs", JobLocation("San Francisco", "CA", "USA", 94103), JobType.FULL_TIME, 120000)
print(job)

# Get job
job = unified_manager.get_job(alice_session, job.job_id)
print(job)

# User applies for the job
application = unified_manager.apply_for_job(alice_session, job.job_id)
print(application)

# Fetch applications
print(unified_manager.get_job_applications(google_session, job.job_id))

# Get user applications
print(unified_manager.get_user_applications(alice_session))

# Update application status
unified_manager.update_application_status(google_session, application.application_id, JobApplicationStatus.ACCEPTED)

# Fetch applications
print(unified_manager.get_job_applications(google_session, job.job_id))

# Remove job
unified_manager.remove_job(google_session, job.job_id)

# Post job
unified_manager.post_job(google_session, "Software Engineer", "Develop backend services", JobLocation("San Francisco", "CA", "USA", 94103), JobType.FULL_TIME, 120000)

# Perform a search
search_results = unified_manager.search(alice_session, "Engineer", SearchType.JOB)

# Display search results
for result, score in search_results:
    print(f"Found: {result.title} (Relevance Score: {score})")
    
# Perform a search
search_results = unified_manager.search(alice_session, "Google", SearchType.COMPANY)

# Display search results
for result, score in search_results:
    print(f"Found: {result.name} (Relevance Score: {score})")

# Perform a search
search_results = unified_manager.search(alice_session, "Bob", SearchType.USER)

# Display search results
for result, score in search_results:
    print(f"Found: {result.name} (Relevance Score: {score})")


# Logout Employer
unified_manager.logout_user(google_session)

# Logout user
unified_manager.logout_user(alice_session)


✅ Alice has logged in successfully!
✅ Bob has logged in successfully!
✅ Alice's profile updated successfully!
{'Headline': 'Software Engineer', 'Summary': 'Experienced in Python, Django, and AI', 'Experience': [], 'Education': [], 'Skills': []}
[Notification(Connection Request: Alice has sent you a connection request. - 🔔 Unread, 2025-02-23 14:55:24.616307)]
[ConnectionRequest(Alice -> Bob, Status=Pending)]
set()
{Alice}
{Bob}
Bob's unread messages: 0
[Notification(Connection Request: Alice has sent you a connection request. - 🔔 Unread, 2025-02-23 14:55:24.616307), Notification(Message: You have a new message from Alice. - 🔔 Unread, 2025-02-23 14:55:24.616734)]
Bob's unread messages: 1
Message(Alice -> Bob, Status=Read, Time=2025-02-23 14:55:24.616720)
✅ Google has logged in successfully!
✅ Google's company profile updated successfully!
{'Headline': 'Tech Firm', 'Summary': 'Technology based firm', 'Company Name': 'Google', 'Founded': 2012, 'Posted Jobs': []}
Job(Backend Developer at Sa